# COMPARACIÓN AUTOMATIZADA DE PRECIOS DE PRODUCTOS DE PRIMERA NECESIDAD EN LOS 3 SUPERMERCADOS MÁS POPULARES: METRO, PLAZA VEA, TOTTUS.

### DESCRIPCIÓN DEL CASO

Se quiere automatizar la comparación de precios de productos básicos de la canasta familiar
(arroz, azúcar, aceite, leche, etc) de tres supermercados que ofrecen sus productos a través de
su propio portal web. Estos supermercados son: Metro, Plaza Vea y Tottus.
Por lo que se pide escribir un programa en Python utilizando Jupyter notebook que recopile los
precios de un producto de la canasta básica familiar de 3 Supermercados anteriormente
mencionados y reporte todos sus artículos relacionados con la palabra clave ingresada en la
búsqueda, para que posteriormente pueda comparar los precios en los distintos
Supermercados de los productos exactamente iguales. Además, que se envíe
automáticamente al correo del usuario-consultor un reporte de precio más alto, precio más
bajo, productos en común comparando los precios entre los supermercados coincidentes.

# Importar librerías


In [249]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pymongo import MongoClient
from tqdm import tqdm
from termcolor import colored
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
import time
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

# Documentación de la nueva actualización de Selenium


In [250]:
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"

# find_element(By.ID, "id")
# find_element(By.NAME, "name")
# find_element(By.XPATH, "xpath")
# find_element(By.LINK_TEXT, "link text")
# find_element(By.PARTIAL_LINK_TEXT, "partial link text")
# find_element(By.TAG_NAME, "tag name")
# find_element(By.CLASS_NAME, "class name")
# find_element(By.CSS_SELECTOR, "css selector")

# Programa para obtener los productos y precios de supermercados Metro (Usuario Jorge)

In [251]:
path = ('C:/Users/jorge/Documents/TRABAJO FINAL/chromedriver.exe')  # Dirección del ejecutable chromedriver.exe
urls='https://www.metro.pe'          # URL de la pagina de Metro                   
options = webdriver.ChromeOptions()   
options.add_argument('--incognito')  # Opcion de abrir ventana de Chrome en modo incognito
s = Service(path)                       
driver = webdriver.Chrome(service=s, options = options)
driver.maximize_window()
driver.get(urls)

time.sleep(3) # lapso de 3 segundo para que la pagina cargue los contenidos

In [252]:
# Hacer clic en "cerrar", pagina emergente que sale en inicio
driver.find_element(By.XPATH, '/html/body/div[46]/div/div[3]/button[1]').click()
time.sleep(1)

In [253]:
# Hacer clic en la barra de búsqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').click()
time.sleep(1)

In [254]:
# Ingresar el producto en la barra de busqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').send_keys('leche')
time.sleep(1)

In [255]:
# Dar clic en ver todos los resultados.
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[2]/div[2]/a/span').click()
time.sleep(1)

In [271]:
# Cantidad de productos
page=driver.find_element(By.XPATH, '/html/body/div[27]/div/div[2]/div[1]/div[1]/span[2]').text
int(page[:3])

307

In [283]:
# Automatización de scrolls (se correra las veces que sea necesario)
tiempo = 15 + 1           # Seran 15 scrolls automaticos
while True:               # Mientras esto sea verdad se realizará lo siguiente:
    tiempo -= 1           # Cuenta regresiva, se resta 1 cada ciclo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # Cada segundo se desplazará hasta la parte inferior de la página 
    time.sleep(1)      # Cada scroll automático tendra un timelap de 1 segundo
    if scrolls < 0:     # Cuando llegue a cero segundos:
        time.sleep(3)   # Le damos 3 segundos a que la pagina carge los contenidos hasta el ultimo scroll automatico
        driver.execute_script("window.scrollBy(0,-310)") # Un scroll up (rueda hacia arriba) para mostrar la ultima fila de productos
        elements = driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]') # Encontrar los productos de la pagina de acuerdo al XPATH
        break           # Se cierra el ciclo

        
print(colored('Productos extraidos:','red'))  
print(len(elements))        # Longitud de los elementos (productos) encontrados hasta coindicir con el total de productos

print(colored('\n Se llegó a extraer todos los productos?.','yellow'))  
print(len(elements) == int(page[:3])) # Comparación 

Productos extraidos:
307
Se llegó a extraer todos los productos?.
True


In [287]:
# Vista a todos los elementos a scrapear, desde el primero hasta el ultimo en ser extradio
n = 1
while n < len(elements):  # Ciclo de acuerdo  a la cantidad de elementos extraidos
    listing = driver.find_element(By.XPATH, f'/html/body/div[27]/div/div[2]/div[4]/div[2]/div[2]/div[2]/div/ul/li[{n}]') 
    driver.execute_script("arguments[0].scrollIntoView(true);", listing)  # Se ira a la parte inferior hasta la ultima fila de productos
    n += 1
    

In [289]:
# Almacenar productos y precios 
productos =  []  # lista vacia
precios = []  # lista vacia
for element in tqdm(elements):  # tqdm:Contar iteraciones, indicar el tiempo que tarda por iteración y el tiempo total transcurrido.  
        name = element.find_element(By.XPATH,'.//a[@class="product-item__name"]').text  # Buscar producto en base al XPATH y mostrar el texto
        price = element.find_element(By.XPATH,'.//span[@class="product-prices__value product-prices__value--best-price"]').text # Buscar precio en base al XPATH y mostrar el texto
        productos.append(name)    # Agregar a la lista productos
        precios.append(price)  # Agregar a la lista precios


100%|██████████| 307/307 [00:43<00:00,  6.99it/s]


In [291]:
#Exportacion de productos y precios a un archivo excel 
dic= {'Producto': productos , "Precio": precios}  # Crear diccionario con los values productos y precios (listas)
df = pd.DataFrame(dic , columns= ["Producto", "Precio"] ) # Convertir a dataframe con columnas de etiquetas Producto y Precio
df.to_excel('metro.xlsx', index=False,encoding='cp1252')  # Exportar a excel, sin indices, y el encoding cp1252

# Lectura y muestra de resultados
print(colored('Imprimiendo resultados.','blue'))  
pd.read_excel('metro.xlsx')

Imprimiendo resultados.


,Producto,Precio
0,Chocolate de Leche Snickers 53g,S/. 4.00
1,Chocolate de Leche Toblerone 100g,S/. 12.50
2,Leche Condensada Nestlé Lata 393g,S/. 5.99
3,Leche Entera UHT Laive 900ml,S/. 3.99
4,Leche Light Botella 400 g,S/. 3.99
...,...,...
302,Bebida de Almendra Chocolate Cuisine & Co Caja...,S/. 10.90
303,Ajonjolí Tostado Naturandes,S/10.20
304,Bebida de Almendra Original Cuisine & Co Caja ...,S/. 10.90
305,Mezcla Láctea Pura Vida Pack de 6 unid Lata 39...,S/18.00


# Programa para obtener los productos y precios de supermercados Tottus  (Usuario Christian)

In [304]:
path = ('C:/Users/jorge/Documents/TRABAJO FINAL/chromedriver.exe')
urls='https://www.tottus.com.pe/'
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
s = Service(path)
driver = webdriver.Chrome(service=s, options = options)
driver.maximize_window()
driver.get(urls)
time.sleep(3)

In [305]:
# Configurando click para búsqueda de productos
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').click()
time.sleep(1)

In [306]:
# Ingresando producto 
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').send_keys('leche')
time.sleep(1)

In [307]:
# Buscar producto mediante el click al icono de busqueda
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div[1]/button').click()
time.sleep(1)

In [308]:
pages=driver.find_elements(By.XPATH, './/ul[@class="Pagination link"]')

for i in pages:
    ultima= i.find_element(By.XPATH, './/a[@aria-label="Page 8"]').text
    pagina = int(ultima)

pagina        

8

In [297]:
#busqueda = input()
list= []
n=1
while n <= pagina:
    driver.get(f'https://www.tottus.com.pe/buscar?q=leche&page={n}')
    time.sleep(5)
    lista = driver.find_elements(By.XPATH,'//li[@class="product"]')
    productos =  []
    precios = []
    for element in tqdm(lista):
        name = element.find_element(By.XPATH,'.//h2[@class="jsx-369422875 name title mini"]').text
        price = element.find_element(By.XPATH,'.//span[@class="jsx-4019956066 "]').text
        productos.append(name)
        precios.append(price)

    lista=list.append(lista)
    n += 1

100%|██████████| 8/8 [00:00<00:00, 16.50it/s]


In [298]:
len(list)

8

In [300]:
dic= {'Producto': names,"Precio": precios }
df = pd.DataFrame(dic , columns= ["Producto","Precio"] )
df.to_excel('tottus.xlsx', index=False,encoding='cp1252')

# Lectura
pd.read_excel('tottus.xlsx')

,Producto,Precio
0,TORTA TRES LECHES VAINILLA MEDIANA,s/ 59.9 UN
1,TORTA TRES LECHES LUCUMA MEDIANA,s/ 59.9 UN
2,TORTA TRES LECHES VAINILLA GRANDE,s/ 65.9 UN
3,TORTA TRES LECHES COCO MEDIANA,s/ 59.9 UN
4,CREMA DE LECHE PASTEURIZADA PIAMONTE 236 ML,s/ 6.5 UN
5,BATIDO NUTRICIONAL BENET VAINILLA 400 GR,s/ 47.9 UN
6,BATIDO NUTRICIONAL BENET VAINILLA 800 GR,s/ 89.9 UN
7,BATIDO NUTRICIONAL BENET ENERGÍA 200 GR,s/ 34.9 UN


# Programa para obtener los productos y precios de supermercados Tottus  (Usuario Brian)

In [325]:
url='https://www.plazavea.com.pe'
def busqueda(search):
    "generar url"
    template = 'https://www.plazavea.com.pe/{}'
    search = search.replace('','')
    return template.format(search)
url = busqueda('leche')
print(url)

https://www.plazavea.com.pe/leche


In [326]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
soup = BeautifulSoup(driver.page_source,'html.parser')

In [327]:
def extraerprecio(item):
    #precio
    precio = item.find('div', 'Showcase__salePrice')
    precio1 = precio.text.split()
    nombre=item.find('a','Showcase__name').text
    #descripcion
    descripcion= (precio1, nombre)
    return descripcion

In [328]:
records = []
resultados= soup.find_all('div','Showcase__content')

for item in resultados:
    record = extraerprecio(item)
    if record:
        records.append(record)

In [329]:
def busqueda(busqueda):
    "generar url"
    template = 'https://www.plazavea.com.pe/{}'
    busqueda = busqueda.replace('','')
    url=template.format(search)
    url += '?page={}'
    return url 

In [333]:
def busqueda(search):
    "generar url"
    template = 'https://www.plazavea.com.pe/{}'
    search = search.replace('','')
    return template.format(search)

def main(term):
    driver = webdriver.Chrome()
    driver.maximize_window()
    record = []
    url= busqueda(term)

    for page in range(1,9):
        time.sleep(2)
        driver.get(url.format(page))
        soup= BeautifulSoup(driver.page_source,'html.parser')
        resultados= soup.find_all('div','Showcase__content')

        for item in resultados:
            record = extraerprecio(item)

            if record:
                records.append(record)               

In [ ]:
main('leche')

In [ ]:
with open("plazavea.csv", 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['precio','descripción'])
    writer.writerows(records)
import pandas as pd
df = pd.read_csv('plazavea.csv')
df

In [448]:
path = ('C:/Users/jorge/Documents/TRABAJO FINAL/chromedriver.exe')
urls='https://www.plazavea.com.pe'
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
s = Service(path)
driver = webdriver.Chrome(service=s, options = options)
driver.maximize_window()
driver.get(urls)
time.sleep(3)

In [449]:
driver.find_element(By.XPATH, '/html/body/div[12]/div/div/div[2]/button[2]').click()

In [450]:
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div/div/input').click()

In [451]:
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div[2]/div[1]/input').send_keys("leche" + Keys.ENTER)

In [452]:
# Cantidad de productos
page=driver.find_element(By.XPATH, '/html/body/div[4]/main/div/div[1]/div[2]/div[3]/div/div[1]/div/h2/span[2]').text
int(page[1:4])

203

In [453]:
tiempo = 5 + 1           # Seran 15 scrolls automaticos
while True:               # Mientras esto sea verdad se realizará lo siguiente:
    tiempo -= 1           # Cuenta regresiva, se resta 1 cada ciclo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # Cada segundo se desplazará hasta la parte inferior de la página 
    time.sleep(3)      # Cada scroll automático tendra un timelap de 1 segundo
    if scrolls < 0:     # Cuando llegue a cero segundos:
        time.sleep(3)   # Le damos 3 segundos a que la pagina carge los contenidos hasta el ultimo scroll automatico
        driver.execute_script("window.scrollBy(0,-500)") # Un scroll up (rueda hacia arriba) para mostrar la ultima fila de productos
        elementos = driver.find_elements(By.XPATH,'//div[@class="HA Showcase Showcase--food"]') # Encontrar los productos de la pagina de acuerdo al XPATH
        break           # Se cierra el ciclo
len(elementos)        

49

In [454]:
productos =  []  # lista vacia
precios = []  # lista vacia
for element in tqdm(elementos):  # tqdm:Contar iteraciones, indicar el tiempo que tarda por iteración y el tiempo total transcurrido.  
        name = element.find_element(By.XPATH,'.//div[@class="Showcase__details__text"]').text  # Buscar producto en base al XPATH y mostrar el texto
        price = element.find_element(By.XPATH,'.//span[@class="price"]').text # Buscar precio en base al XPATH y mostrar el texto
        productos.append(name)    # Agregar a la lista productos
        precios.append(price)  # Agregar a la lista precios

100%|██████████| 49/49 [00:03<00:00, 15.27it/s]


In [455]:
dic= {'Producto': productos,"Precio": precios }
df = pd.DataFrame(dic , columns= ["Producto","Precio"] )
df.to_excel('plazavea.xlsx', index=False,encoding='cp1252')

# Lectura
pd.read_excel('plazavea.xlsx')

,Producto,Precio
0,Leche Evaporada GLORIA Light Lata 400g Paquete...,S/ 21.60 un
1,Leche Evaporada GLORIA sin Lactosa Lata 400g P...,S/ 22.80 un
2,Leche GLORIA Evaporada Entera Lata 400g Paquet...,S/ 22.90 un
3,Leche GLORIA Evaporada Niños Lata 400g Paquete...,S/ 23.40 un
4,Leche Sin Lactosa LAIVE Botella 400g Paquete 6...,S/ 21.90 un
5,Leche GLORIA UHT Entera Bolsa 900ml Paquete 3u...,S/ 11.70 un
6,Leche UHT GLORIA Light Pack 3un x 1L\nPaquete 3un,S/ 15.70 un
7,Leche UHT LAIVE Sin Lactosa Caja 1L Paquete 4u...,S/ 20.30 un
8,Leche UHT GLORIA Entera Pack 3un x 1L\nPaquete...,S/ 15.10 un
9,Leche UHT GLORIA sin Lactosa Pack 3un x 1L\nPa...,S/ 16.20 un


In [392]:
# Volver al inicio
driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/header/div[2]/div/div[2]/a').click()

# Envío de correo automático (FASE de prueba)

In [204]:
import smtplib  # Importar la librería

de_email ="okoyekennethoptimizer@gmail.com"   # Correo de envio (emisor)
contraseña = "nmmrjdphptwfgjgb"               # contraseña del emisor
para_email = "20200339@LAMOLINA.EDU.PE"       # Email de destino
asunto = "TRABAJO FINAL LP2"                  # Asunto de llegada
text ="""                                    
Hola estimada profesora, le enviamos las comparaciones de los productos.
Codigo de ntegrantes: 
"""                                           # Contenido del mensaje

mensaje = f'Subject: {asunto}\n\n{text}\n\n'  # Formato de llegada del mensaje
connection = smtplib.SMTP("smtp.gmail.com", port= 587) # Conexión al dominio gmail.com con puerto = 587
connection.starttls()             # Empezar  conexión            
connection.login(user=de_email, password=contraseña) # Logeando con correo y contraseña
connection.sendmail(from_addr=de_email, to_addrs=para_email, msg= mensaje) # Enviar email a destinatario
connection.close()                # Cerrar conexión    